In [5]:
from models.Payload import RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date

#data
sec = SecApi()
sec.request('affected_detail',SecPayload())
df=sec.dataframe(FECHA_INT_STR=str2date)
df= df[df['NOMBRE_REGION']=='Valparaiso'].sort_values(by=['FECHA_INT_STR'],ascending=False)
df


ImportError: cannot import name 'str2date' from 'utils.transforms' (c:\Users\crcam\Dev\sec-report\utils\transforms.py)